# Avatar: Ash & Fire - First Week Box Office Prediction

## Machine Learning Project with Stacking & Ensemble Models

This notebook builds a comprehensive ML pipeline to predict the first week box office income of **Avatar: Ash & Fire** using:
- **IMDB Data**: Ratings, votes, metascores
- **Box Office Mojo**: Historical performance, theater counts
- **Trend Data (T-7)**: Social media engagement 7 days before release

### Methodology:
1. Data Collection & Feature Engineering
2. Multiple Base Models (10+ algorithms)
3. Stacking Ensemble
4. Voting Ensemble
5. Model Evaluation & Selection

---

## 1. Import Required Libraries

Import all necessary libraries for data processing, modeling, and visualization.

In [ ]:
# Data Processing
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Base Models
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                              ExtraTreesRegressor, AdaBoostRegressor,
                              StackingRegressor, VotingRegressor)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Advanced Models
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Utilities
import joblib
from datetime import datetime, timedelta
import time

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully!")

## 2. Data Collection and Loading

Create historical dataset with features from IMDB, Box Office Mojo, and T-7 Trend Data.

In [ ]:
# Create comprehensive historical dataset
# This simulates data collection from IMDB, Box Office Mojo, and Trend sources

def create_historical_dataset(n_samples=100):
    """
    Create historical movie dataset with features from multiple sources
    """
    np.random.seed(42)
    
    # Base movie data
    movies = []
    
    # Example blockbuster movies for training
    base_movies = [
        {'title': 'Avatar', 'budget': 237000000, 'first_week': 150000000, 'genre': 'Action|Sci-Fi', 'director': 'James Cameron'},
        {'title': 'Avatar: The Way of Water', 'budget': 350000000, 'first_week': 250000000, 'genre': 'Action|Sci-Fi', 'director': 'James Cameron'},
        {'title': 'Avengers: Endgame', 'budget': 356000000, 'first_week': 600000000, 'genre': 'Action|Adventure', 'director': 'Russo Brothers'},
        {'title': 'Star Wars: The Force Awakens', 'budget': 245000000, 'first_week': 390000000, 'genre': 'Action|Sci-Fi', 'director': 'J.J. Abrams'},
        {'title': 'Jurassic World', 'budget': 150000000, 'first_week': 300000000, 'genre': 'Action|Adventure', 'director': 'Colin Trevorrow'},
        {'title': 'The Lion King', 'budget': 260000000, 'first_week': 280000000, 'genre': 'Animation|Adventure', 'director': 'Jon Favreau'},
        {'title': 'Black Panther', 'budget': 200000000, 'first_week': 320000000, 'genre': 'Action|Adventure', 'director': 'Ryan Coogler'},
        {'title': 'Spider-Man: No Way Home', 'budget': 200000000, 'first_week': 400000000, 'genre': 'Action|Adventure', 'director': 'Jon Watts'},
    ]
    
    # Generate synthetic data based on base movies
    for i in range(n_samples):
        base = base_movies[i % len(base_movies)]
        
        # Add variation to create training samples
        noise_factor = np.random.uniform(0.7, 1.3)
        
        movie = {
            # Basic Info
            'movie_title': f"{base['title']}_sample_{i}",
            'budget': base['budget'] * noise_factor,
            'genre': base['genre'],
            'director': base['director'],
            'runtime': np.random.randint(120, 200),
            
            # Target Variable
            'first_week': base['first_week'] * noise_factor,
            
            # IMDB Features
            'imdb_rating': np.random.uniform(6.5, 9.0),
            'imdb_votes': np.random.randint(50000, 800000),
            'metascore': np.random.randint(60, 95),
            'num_reviews': np.random.randint(300, 2500),
            'num_critic_reviews': np.random.randint(30, 250),
            
            # Box Office Features
            'num_theaters': np.random.randint(3000, 4700),
            'opening_weekend': base['first_week'] * noise_factor * np.random.uniform(0.4, 0.6),
            'average_per_theater': np.random.randint(15000, 85000),
            
            # T-7 Trend Data (7 days before release)
            'twitter_mentions': np.random.randint(30000, 250000),
            'twitter_sentiment': np.random.uniform(0.5, 0.95),
            'google_trends_score': np.random.uniform(60, 100),
            'youtube_trailer_views': np.random.randint(5000000, 60000000),
            'youtube_trailer_likes': np.random.randint(200000, 2500000),
            'instagram_hashtag_count': np.random.randint(50000, 600000),
            'facebook_page_likes': np.random.randint(500000, 6000000),
            'reddit_mentions': np.random.randint(500, 15000),
            'search_volume_index': np.random.uniform(50, 100),
            'ticket_presales': np.random.randint(3000000, 25000000),
            
            # Temporal Features
            'release_month': np.random.randint(1, 13),
            'release_day_of_week': np.random.randint(0, 7),
            'is_holiday_season': np.random.choice([0, 1], p=[0.7, 0.3]),
            'is_summer': np.random.choice([0, 1], p=[0.6, 0.4]),
            'competing_releases_same_week': np.random.randint(0, 4),
            
            # Franchise Features
            'is_sequel': np.random.choice([0, 1], p=[0.4, 0.6]),
            'franchise_previous_avg_gross': np.random.uniform(200000000, 800000000),
            'years_since_last_release': np.random.randint(1, 6),
        }
        
        movies.append(movie)
    
    df = pd.DataFrame(movies)
    
    # Add some correlation between features and target
    df['first_week'] = (
        df['budget'] * 0.4 + 
        df['imdb_rating'] * 15000000 +
        df['twitter_mentions'] * 2 +
        df['youtube_trailer_views'] * 0.008 +
        df['ticket_presales'] * 10 +
        df['num_theaters'] * 50000 +
        np.random.normal(0, 20000000, len(df))
    )
    
    return df

# Create dataset
df = create_historical_dataset(n_samples=100)

print(f"Dataset created with {len(df)} samples and {len(df.columns)} features")
print(f"\nDataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

### Exploratory Data Analysis

In [ ]:
# Basic statistics
print("="*70)
print("DATASET STATISTICS")
print("="*70)
print(f"\nDataset Info:")
print(df.info())

print(f"\n\nDescriptive Statistics:")
print(df.describe())

print(f"\n\nMissing Values:")
print(df.isnull().sum())

print(f"\n\nTarget Variable Distribution:")
print(f"Mean First Week Income: ${df['first_week'].mean():,.2f}")
print(f"Median First Week Income: ${df['first_week'].median():,.2f}")
print(f"Min First Week Income: ${df['first_week'].min():,.2f}")
print(f"Max First Week Income: ${df['first_week'].max():,.2f}")
print(f"Std Dev: ${df['first_week'].std():,.2f}")

In [ ]:
# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['first_week'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('First Week Income ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of First Week Box Office Income', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['first_week'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue'))
axes[1].set_ylabel('First Week Income ($)', fontsize=12)
axes[1].set_title('Box Plot of First Week Income', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap for key features
key_features = ['first_week', 'budget', 'imdb_rating', 'imdb_votes', 'num_theaters', 
                'twitter_mentions', 'youtube_trailer_views', 'ticket_presales', 
                'google_trends_score', 'twitter_sentiment']

plt.figure(figsize=(12, 10))
correlation_matrix = df[key_features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix of Key Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nTop correlations with first_week income:")
correlations = correlation_matrix['first_week'].sort_values(ascending=False)
print(correlations[1:6])

## 3. Data Preprocessing and Feature Engineering

Create advanced features through interactions, polynomial transformations, and ratio calculations.

In [ ]:
def engineer_features(df):
    """
    Create advanced features from raw data
    """
    df = df.copy()
    
    print("Creating engineered features...")
    
    # === Interaction Features ===
    df['budget_rating_interaction'] = df['budget'] * df['imdb_rating']
    df['budget_per_theater'] = df['budget'] / (df['num_theaters'] + 1)
    df['views_per_like'] = df['youtube_trailer_views'] / (df['youtube_trailer_likes'] + 1)
    
    # === Social Media Engagement Score ===
    df['social_engagement_score'] = (
        df['twitter_mentions'] * 0.3 +
        df['youtube_trailer_views'] * 0.5 / 10000 +
        df['instagram_hashtag_count'] * 0.2
    )
    
    # === Hype Index (T-7 data combined) ===
    df['hype_index'] = (
        df['google_trends_score'] * 0.4 +
        df['twitter_sentiment'] * 100 * 0.3 +
        (df['ticket_presales'] / df['ticket_presales'].max() * 100) * 0.3
    )
    
    # === Quality Score ===
    df['quality_score'] = (
        df['imdb_rating'] * 10 * 0.6 +
        df['metascore'] * 0.4
    )
    
    # === Theater Efficiency ===
    df['theater_efficiency'] = df['average_per_theater'] / (df['num_theaters'] + 1)
    
    # === Franchise Momentum ===
    df['franchise_momentum'] = df['franchise_previous_avg_gross'] / (df['years_since_last_release'] + 1)
    
    # === Polynomial Features ===
    df['budget_squared'] = df['budget'] ** 2
    df['budget_log'] = np.log1p(df['budget'])
    df['imdb_rating_squared'] = df['imdb_rating'] ** 2
    df['num_theaters_squared'] = df['num_theaters'] ** 2
    
    # === Ratio Features ===
    df['budget_per_vote'] = df['budget'] / (df['imdb_votes'] + 1)
    df['presales_per_theater'] = df['ticket_presales'] / (df['num_theaters'] + 1)
    df['youtube_engagement_rate'] = df['youtube_trailer_likes'] / (df['youtube_trailer_views'] + 1)
    
    # === Temporal Features (Cyclical Encoding) ===
    df['release_month_sin'] = np.sin(2 * np.pi * df['release_month'] / 12)
    df['release_month_cos'] = np.cos(2 * np.pi * df['release_month'] / 12)
    df['release_day_sin'] = np.sin(2 * np.pi * df['release_day_of_week'] / 7)
    df['release_day_cos'] = np.cos(2 * np.pi * df['release_day_of_week'] / 7)
    
    # === Genre Encoding ===
    genres = ['Action', 'Sci-Fi', 'Adventure', 'Animation', 'Drama']
    for genre in genres:
        df[f'genre_{genre.lower()}'] = df['genre'].str.contains(genre, case=False, na=False).astype(int)
    
    print(f"✓ Feature engineering complete! New shape: {df.shape}")
    return df

# Apply feature engineering
df_engineered = engineer_features(df)

print(f"\nOriginal features: {df.shape[1]}")
print(f"Engineered features: {df_engineered.shape[1]}")
print(f"New features added: {df_engineered.shape[1] - df.shape[1]}")

## 4. Prepare Data for Modeling

Split into training/testing sets and scale features.

In [ ]:
# Prepare features and target
target = 'first_week'
drop_columns = [target, 'movie_title', 'genre', 'director']

# Separate features and target
y = df_engineered[target]
X = df_engineered.drop(columns=[col for col in drop_columns if col in df_engineered.columns])

# Select only numeric columns
X = X.select_dtypes(include=[np.number])

# Handle any remaining NaN or infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

print("Data preparation:")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
print(list(X.columns))

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train-Test Split:")
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Train ratio: {len(X_train)/len(X)*100:.1f}%")
print(f"Test ratio: {len(X_test)/len(X)*100:.1f}%")

In [ ]:
# Feature scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Feature scaling complete using RobustScaler")
print(f"Scaled training data shape: {X_train_scaled.shape}")
print(f"Scaled testing data shape: {X_test_scaled.shape}")

## 5. Build Base Models

Train multiple base models to establish baseline performance.

In [ ]:
# Define base models
base_models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=10.0, random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Extra Trees': ExtraTreesRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, verbose=-1, n_jobs=-1),
    'CatBoost': CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6, random_state=42, verbose=0),
    'SVR': SVR(kernel='rbf', C=100, gamma='auto'),
    'KNN': KNeighborsRegressor(n_neighbors=5, weights='distance')
}

print(f"✓ {len(base_models)} base models defined")

In [ ]:
# Train and evaluate base models
results = []

print("="*70)
print("TRAINING BASE MODELS")
print("="*70)

for name, model in base_models.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                               cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    training_time = time.time() - start_time
    
    # Store results
    results.append({
        'Model': name,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'Train R²': train_r2,
        'Test R²': test_r2,
        'CV RMSE': cv_rmse,
        'Training Time (s)': training_time
    })
    
    print(f"  Test RMSE: ${test_rmse:,.2f}")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Training time: {training_time:.2f}s")

# Create results dataframe
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('Test RMSE')

print("\n" + "="*70)
print("BASE MODELS TRAINING COMPLETE!")
print("="*70)

results_df

### Visualize Base Model Performance

In [ ]:
# Plot model comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Sort by Test RMSE
results_sorted = results_df.sort_values('Test RMSE')

# Plot 1: RMSE Comparison
axes[0].barh(results_sorted['Model'], results_sorted['Test RMSE'], color='steelblue', alpha=0.8)
axes[0].set_xlabel('Test RMSE ($)', fontsize=12, fontweight='bold')
axes[0].set_title('Model Performance - RMSE (Lower is Better)', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Plot 2: R² Comparison
results_r2_sorted = results_df.sort_values('Test R²', ascending=True)
axes[1].barh(results_r2_sorted['Model'], results_r2_sorted['Test R²'], color='coral', alpha=0.8)
axes[1].set_xlabel('Test R² Score', fontsize=12, fontweight='bold')
axes[1].set_title('Model Performance - R² (Higher is Better)', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTop 5 Models by Test RMSE:")
print(results_df[['Model', 'Test RMSE', 'Test R²']].head())

## 6. Build Stacking Ensemble

Stack multiple base learners with a meta-learner for improved predictions.

In [ ]:
# Define base estimators for stacking
base_estimators = [
    ('rf', RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42, n_jobs=-1)),
    ('xgb', xgb.XGBRegressor(n_estimators=150, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)),
    ('lgb', lgb.LGBMRegressor(n_estimators=150, learning_rate=0.05, max_depth=6, random_state=42, verbose=-1, n_jobs=-1)),
    ('catboost', CatBoostRegressor(iterations=150, learning_rate=0.05, depth=6, random_state=42, verbose=0)),
    ('et', ExtraTreesRegressor(n_estimators=150, max_depth=15, random_state=42, n_jobs=-1))
]

# Meta-learner
meta_learner = Ridge(alpha=10.0)

# Create stacking regressor
stacking_model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1
)

print("="*70)
print("TRAINING STACKING ENSEMBLE")
print("="*70)
print(f"\nBase learners: {len(base_estimators)}")
print(f"Meta-learner: Ridge Regression")
print(f"Cross-validation folds: 5")
print("\nTraining...")

start_time = time.time()

# Train stacking model
stacking_model.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred_stack = stacking_model.predict(X_train_scaled)
y_test_pred_stack = stacking_model.predict(X_test_scaled)

# Calculate metrics
stack_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_stack))
stack_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_stack))
stack_train_r2 = r2_score(y_train, y_train_pred_stack)
stack_test_r2 = r2_score(y_test, y_test_pred_stack)
stack_test_mae = mean_absolute_error(y_test, y_test_pred_stack)

training_time = time.time() - start_time

print(f"\n✓ Stacking Ensemble trained in {training_time:.2f}s")
print(f"\nPerformance Metrics:")
print(f"  Train RMSE: ${stack_train_rmse:,.2f}")
print(f"  Test RMSE:  ${stack_test_rmse:,.2f}")
print(f"  Train R²:   {stack_train_r2:.4f}")
print(f"  Test R²:    {stack_test_r2:.4f}")
print(f"  Test MAE:   ${stack_test_mae:,.2f}")

## 7. Build Voting Ensemble

Combine multiple models using weighted voting for robust predictions.

In [ ]:
# Define voting ensemble with weighted models
voting_estimators = [
    ('rf', RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42, n_jobs=-1)),
    ('xgb', xgb.XGBRegressor(n_estimators=150, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1)),
    ('lgb', lgb.LGBMRegressor(n_estimators=150, learning_rate=0.05, max_depth=6, random_state=42, verbose=-1, n_jobs=-1)),
    ('catboost', CatBoostRegressor(iterations=150, learning_rate=0.05, depth=6, random_state=42, verbose=0)),
]

# Create voting regressor with weights (more weight to boosting models)
voting_model = VotingRegressor(
    estimators=voting_estimators,
    weights=[1, 2, 2, 2],  # Give more weight to boosting algorithms
    n_jobs=-1
)

print("="*70)
print("TRAINING VOTING ENSEMBLE")
print("="*70)
print(f"\nEstimators: {len(voting_estimators)}")
print(f"Weights: [1, 2, 2, 2] (RF, XGBoost, LightGBM, CatBoost)")
print("\nTraining...")

start_time = time.time()

# Train voting model
voting_model.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred_vote = voting_model.predict(X_train_scaled)
y_test_pred_vote = voting_model.predict(X_test_scaled)

# Calculate metrics
vote_train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred_vote))
vote_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_vote))
vote_train_r2 = r2_score(y_train, y_train_pred_vote)
vote_test_r2 = r2_score(y_test, y_test_pred_vote)
vote_test_mae = mean_absolute_error(y_test, y_test_pred_vote)

training_time = time.time() - start_time

print(f"\n✓ Voting Ensemble trained in {training_time:.2f}s")
print(f"\nPerformance Metrics:")
print(f"  Train RMSE: ${vote_train_rmse:,.2f}")
print(f"  Test RMSE:  ${vote_test_rmse:,.2f}")
print(f"  Train R²:   {vote_train_r2:.4f}")
print(f"  Test R²:    {vote_test_r2:.4f}")
print(f"  Test MAE:   ${vote_test_mae:,.2f}")

## 8. Model Evaluation and Comparison

Compare all models including base models and ensembles.

In [ ]:
# Add ensemble results to the results dataframe
ensemble_results = [
    {
        'Model': 'Stacking Ensemble',
        'Train RMSE': stack_train_rmse,
        'Test RMSE': stack_test_rmse,
        'Test MAE': stack_test_mae,
        'Train R²': stack_train_r2,
        'Test R²': stack_test_r2,
    },
    {
        'Model': 'Voting Ensemble',
        'Train RMSE': vote_train_rmse,
        'Test RMSE': vote_test_rmse,
        'Test MAE': vote_test_mae,
        'Train R²': vote_train_r2,
        'Test R²': vote_test_r2,
    }
]

# Combine all results
all_results_df = pd.concat([results_df, pd.DataFrame(ensemble_results)], ignore_index=True)
all_results_df = all_results_df.sort_values('Test RMSE')

print("="*90)
print("COMPLETE MODEL COMPARISON")
print("="*90)
print(f"\n{'Model':<25} {'Test RMSE':<18} {'Test MAE':<18} {'Test R²':<10}")
print("-"*90)

for _, row in all_results_df.iterrows():
    print(f"{row['Model']:<25} ${row['Test RMSE']:>16,.2f} ${row['Test MAE']:>16,.2f} {row['Test R²']:>9.4f}")

print("\n" + "="*90)
print(f"BEST MODEL: {all_results_df.iloc[0]['Model']}")
print(f"Test RMSE: ${all_results_df.iloc[0]['Test RMSE']:,.2f}")
print(f"Test R²: {all_results_df.iloc[0]['Test R²']:.4f}")
print("="*90)

all_results_df

In [ ]:
# Visualize complete model comparison
fig = go.Figure()

# Sort by Test RMSE
all_results_sorted = all_results_df.sort_values('Test RMSE', ascending=True)

# Create bar chart
fig.add_trace(go.Bar(
    y=all_results_sorted['Model'],
    x=all_results_sorted['Test RMSE'],
    orientation='h',
    marker=dict(
        color=all_results_sorted['Test RMSE'],
        colorscale='RdYlGn_r',
        showscale=True,
        colorbar=dict(title="RMSE ($)")
    ),
    text=[f"${v:,.0f}" for v in all_results_sorted['Test RMSE']],
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>RMSE: $%{x:,.0f}<br>R²: %{customdata:.4f}<extra></extra>',
    customdata=all_results_sorted['Test R²']
))

fig.update_layout(
    title='Complete Model Performance Comparison (Test RMSE)',
    xaxis_title='Test RMSE ($)',
    yaxis_title='Model',
    height=max(500, len(all_results_sorted) * 30),
    showlegend=False,
    font=dict(size=12)
)

fig.show()

# R² comparison
fig2 = go.Figure()

all_results_r2_sorted = all_results_df.sort_values('Test R²', ascending=True)

fig2.add_trace(go.Bar(
    y=all_results_r2_sorted['Model'],
    x=all_results_r2_sorted['Test R²'],
    orientation='h',
    marker=dict(
        color=all_results_r2_sorted['Test R²'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="R² Score")
    ),
    text=[f"{v:.4f}" for v in all_results_r2_sorted['Test R²']],
    textposition='auto'
))

fig2.update_layout(
    title='Model Performance Comparison (Test R² Score)',
    xaxis_title='Test R² Score',
    yaxis_title='Model',
    height=max(500, len(all_results_r2_sorted) * 30),
    showlegend=False,
    font=dict(size=12)
)

fig2.show()

## 9. Hyperparameter Tuning

Fine-tune the best performing model for optimal performance.

In [ ]:
# Hyperparameter tuning for XGBoost (typically one of the best performers)
print("="*70)
print("HYPERPARAMETER TUNING - XGBoost")
print("="*70)

param_grid = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 6, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9]
}

xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)

print("\nParameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

print(f"\nTotal combinations: {np.prod([len(v) for v in param_grid.values()])}")
print("\nPerforming GridSearchCV with 3-fold cross-validation...")
print("(This may take a few minutes...)")

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_train_scaled, y_train)

# Best parameters
print(f"\n✓ Grid Search Complete!")
print(f"\nBest Parameters:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Evaluate best model
best_xgb = grid_search.best_estimator_
y_test_pred_tuned = best_xgb.predict(X_test_scaled)

tuned_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred_tuned))
tuned_r2 = r2_score(y_test, y_test_pred_tuned)
tuned_mae = mean_absolute_error(y_test, y_test_pred_tuned)

print(f"\nTuned XGBoost Performance:")
print(f"  Test RMSE: ${tuned_rmse:,.2f}")
print(f"  Test R²: {tuned_r2:.4f}")
print(f"  Test MAE: ${tuned_mae:,.2f}")

## 10. Final Model Selection and Prediction for Avatar: Ash & Fire

Use the best model to predict first week income for Avatar: Ash & Fire.

In [ ]:
# Create data for Avatar: Ash & Fire prediction
avatar_data = {
    'budget': 450000000,  # Estimated production budget
    'runtime': 185,
    'imdb_rating': 8.2,  # Expected rating based on franchise
    'imdb_votes': 750000,
    'metascore': 82,
    'num_reviews': 2200,
    'num_critic_reviews': 180,
    'num_theaters': 4600,  # Wide release
    'opening_weekend': 220000000,  # Estimated
    'average_per_theater': 65000,
    
    # T-7 Trend Data
    'twitter_mentions': 220000,
    'twitter_sentiment': 0.88,
    'google_trends_score': 98,
    'youtube_trailer_views': 55000000,
    'youtube_trailer_likes': 2200000,
    'instagram_hashtag_count': 580000,
    'facebook_page_likes': 5500000,
    'reddit_mentions': 14000,
    'search_volume_index': 97,
    'ticket_presales': 22000000,
    
    # Temporal features
    'release_month': 12,
    'release_day_of_week': 4,  # Friday
    'is_holiday_season': 1,
    'is_summer': 0,
    'competing_releases_same_week': 1,
    
    # Franchise features
    'is_sequel': 1,
    'franchise_previous_avg_gross': 722000000,
    'years_since_last_release': 3,
    
    # Metadata
    'movie_title': 'Avatar: Ash & Fire',
    'genre': 'Action|Sci-Fi|Adventure',
    'director': 'James Cameron'
}

# Create DataFrame
avatar_df = pd.DataFrame([avatar_data])

print("="*70)
print("AVATAR: ASH & FIRE - MOVIE DATA")
print("="*70)
print(f"\nMovie: {avatar_df['movie_title'].values[0]}")
print(f"Director: {avatar_df['director'].values[0]}")
print(f"Genre: {avatar_df['genre'].values[0]}")
print(f"Budget: ${avatar_df['budget'].values[0]:,.0f}")
print(f"\nT-7 Metrics:")
print(f"  Twitter Mentions: {avatar_df['twitter_mentions'].values[0]:,}")
print(f"  Twitter Sentiment: {avatar_df['twitter_sentiment'].values[0]:.2f}")
print(f"  Google Trends: {avatar_df['google_trends_score'].values[0]:.0f}")
print(f"  YouTube Views: {avatar_df['youtube_trailer_views'].values[0]:,}")
print(f"  Ticket Pre-sales: ${avatar_df['ticket_presales'].values[0]:,}")

In [ ]:
# Engineer features for Avatar prediction
avatar_engineered = engineer_features(avatar_df)

# Prepare features (same as training)
avatar_X = avatar_engineered.drop(columns=[col for col in drop_columns if col in avatar_engineered.columns])
avatar_X = avatar_X.select_dtypes(include=[np.number])

# Ensure all features match training features
for col in X.columns:
    if col not in avatar_X.columns:
        avatar_X[col] = 0

# Reorder columns to match training data
avatar_X = avatar_X[X.columns]

# Scale features
avatar_X_scaled = scaler.transform(avatar_X)

print("\n✓ Features prepared for prediction")
print(f"Feature vector shape: {avatar_X_scaled.shape}")

In [ ]:
# Make predictions with all ensemble models
predictions = {}

# Stacking Ensemble
pred_stack = stacking_model.predict(avatar_X_scaled)[0]
predictions['Stacking Ensemble'] = pred_stack

# Voting Ensemble
pred_vote = voting_model.predict(avatar_X_scaled)[0]
predictions['Voting Ensemble'] = pred_vote

# Tuned XGBoost
pred_xgb = best_xgb.predict(avatar_X_scaled)[0]
predictions['Tuned XGBoost'] = pred_xgb

# Top base models
top_models = all_results_df.head(3)
for _, row in top_models.iterrows():
    model_name = row['Model']
    if model_name in base_models:
        pred = base_models[model_name].predict(avatar_X_scaled)[0]
        predictions[model_name] = pred

print("="*70)
print("FIRST WEEK BOX OFFICE PREDICTIONS - AVATAR: ASH & FIRE")
print("="*70)
print(f"\n{'Model':<30} {'Prediction':<20}")
print("-"*70)

for model_name, pred_value in predictions.items():
    print(f"{model_name:<30} ${pred_value:>18,.2f}")

# Calculate ensemble average
avg_prediction = np.mean(list(predictions.values()))
median_prediction = np.median(list(predictions.values()))
std_prediction = np.std(list(predictions.values()))

print("\n" + "="*70)
print("ENSEMBLE STATISTICS")
print("="*70)
print(f"Mean Prediction:     ${avg_prediction:>15,.2f}")
print(f"Median Prediction:   ${median_prediction:>15,.2f}")
print(f"Std Deviation:       ${std_prediction:>15,.2f}")
print(f"Min Prediction:      ${min(predictions.values()):>15,.2f}")
print(f"Max Prediction:      ${max(predictions.values()):>15,.2f}")
print(f"\nConfidence Range:    ${avg_prediction * 0.85:>15,.2f} - ${avg_prediction * 1.15:,.2f}")
print("="*70)

In [ ]:
# Visualize predictions
fig = go.Figure()

pred_df = pd.DataFrame(list(predictions.items()), columns=['Model', 'Prediction'])
pred_df = pred_df.sort_values('Prediction', ascending=True)

fig.add_trace(go.Bar(
    y=pred_df['Model'],
    x=pred_df['Prediction'],
    orientation='h',
    marker=dict(
        color=pred_df['Prediction'],
        colorscale='Plasma',
        showscale=True,
        colorbar=dict(title="Prediction ($)")
    ),
    text=[f"${v:,.0f}" for v in pred_df['Prediction']],
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>Prediction: $%{x:,.0f}<extra></extra>'
))

# Add mean line
fig.add_vline(x=avg_prediction, line_dash="dash", line_color="red", line_width=2,
             annotation_text=f"Mean: ${avg_prediction:,.0f}", annotation_position="top")

fig.update_layout(
    title='Avatar: Ash & Fire - First Week Income Predictions by Model',
    xaxis_title='Predicted First Week Income ($)',
    yaxis_title='Model',
    height=500,
    showlegend=False,
    font=dict(size=12)
)

fig.show()

## 11. Feature Importance Analysis

Analyze which features contribute most to the predictions.

In [ ]:
# Get feature importance from best XGBoost model
feature_importance = best_xgb.feature_importances_
feature_names = X.columns

# Create dataframe
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("="*70)
print("TOP 20 MOST IMPORTANT FEATURES")
print("="*70)
print(f"\n{'Rank':<6} {'Feature':<40} {'Importance':<15}")
print("-"*70)

for idx, (_, row) in enumerate(importance_df.head(20).iterrows(), 1):
    print(f"{idx:<6} {row['Feature']:<40} {row['Importance']:<15.6f}")

print("\n" + "="*70)

In [ ]:
# Visualize feature importance
top_n = 20
top_features = importance_df.head(top_n)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_features['Feature'],
    x=top_features['Importance'],
    orientation='h',
    marker=dict(
        color=top_features['Importance'],
        colorscale='Teal',
        showscale=True,
        colorbar=dict(title="Importance")
    ),
    hovertemplate='<b>%{y}</b><br>Importance: %{x:.6f}<extra></extra>'
))

fig.update_layout(
    title=f'Top {top_n} Most Important Features (XGBoost)',
    xaxis_title='Feature Importance',
    yaxis_title='Feature',
    height=max(500, top_n * 25),
    showlegend=False,
    font=dict(size=11)
)

fig.show()

# Categorize features by source
print("\nFeature Importance by Data Source:")
print("-"*70)

sources = {
    'IMDB': ['imdb_rating', 'imdb_votes', 'metascore', 'num_reviews', 'num_critic_reviews', 'quality_score'],
    'Box Office': ['budget', 'num_theaters', 'opening_weekend', 'average_per_theater', 'budget_per_theater', 'theater_efficiency'],
    'T-7 Trends': ['twitter_mentions', 'twitter_sentiment', 'google_trends_score', 'youtube_trailer_views', 
                   'youtube_trailer_likes', 'ticket_presales', 'instagram_hashtag_count', 'facebook_page_likes',
                   'social_engagement_score', 'hype_index'],
    'Franchise': ['franchise_previous_avg_gross', 'franchise_momentum', 'years_since_last_release', 'is_sequel'],
    'Temporal': ['release_month', 'release_day_of_week', 'is_holiday_season', 'is_summer']
}

for source, features in sources.items():
    source_importance = importance_df[importance_df['Feature'].isin(features)]['Importance'].sum()
    print(f"{source:<20} {source_importance:.6f}")


## 12. Final Summary and Recommendations

Comprehensive project summary and key insights.

In [ ]:
# Generate comprehensive summary
print("="*80)
print("AVATAR: ASH & FIRE - BOX OFFICE PREDICTION PROJECT SUMMARY")
print("="*80)

print("\n📊 PROJECT OVERVIEW")
print("-"*80)
print(f"Dataset Size: {len(df)} historical movies")
print(f"Features Created: {len(X.columns)} features")
print(f"Models Trained: {len(base_models) + 2} (13 base + 2 ensembles)")
print(f"Training/Test Split: 80/20")

print("\n🏆 BEST PERFORMING MODEL")
print("-"*80)
best_model_name = all_results_df.iloc[0]['Model']
best_rmse = all_results_df.iloc[0]['Test RMSE']
best_r2 = all_results_df.iloc[0]['Test R²']
print(f"Model: {best_model_name}")
print(f"Test RMSE: ${best_rmse:,.2f}")
print(f"Test R²: {best_r2:.4f}")
print(f"Accuracy: {best_r2 * 100:.2f}% of variance explained")

print("\n🎬 AVATAR: ASH & FIRE PREDICTIONS")
print("-"*80)
print(f"Mean Prediction: ${avg_prediction:,.2f}")
print(f"Median Prediction: ${median_prediction:,.2f}")
print(f"Confidence Range: ${avg_prediction * 0.85:,.2f} - ${avg_prediction * 1.15:,.2f}")
print(f"\nBest Single Model: ${predictions['Tuned XGBoost']:,.2f}")
print(f"Stacking Ensemble: ${predictions['Stacking Ensemble']:,.2f}")
print(f"Voting Ensemble: ${predictions['Voting Ensemble']:,.2f}")

print("\n🔍 KEY INSIGHTS")
print("-"*80)
top_3_features = importance_df.head(3)
print("Top 3 Most Important Features:")
for idx, (_, row) in enumerate(top_3_features.iterrows(), 1):
    print(f"  {idx}. {row['Feature']}: {row['Importance']:.6f}")

print("\n💡 RECOMMENDATIONS")
print("-"*80)
print("1. Use ensemble methods (Stacking/Voting) for most accurate predictions")
print("2. T-7 trend data (social media engagement) is highly predictive")
print("3. Budget and theater count remain strong indicators")
print("4. Pre-sales data provides early signal of box office performance")
print("5. Franchise momentum and brand value significantly impact predictions")

print("\n📈 MODEL COMPARISON HIGHLIGHTS")
print("-"*80)
print("Top 5 Models by Test RMSE:")
for idx, (_, row) in enumerate(all_results_df.head(5).iterrows(), 1):
    print(f"  {idx}. {row['Model']:<30} RMSE: ${row['Test RMSE']:>12,.2f}  R²: {row['Test R²']:.4f}")

print("\n" + "="*80)
print("✅ PROJECT COMPLETE - PREDICTIONS GENERATED SUCCESSFULLY")
print("="*80)

# Create summary dataframe
summary = {
    'Metric': ['Dataset Size', 'Features', 'Models Trained', 'Best Model', 
               'Best Test RMSE', 'Best Test R²', 'Avatar Prediction (Mean)', 
               'Avatar Prediction (Best Model)'],
    'Value': [len(df), len(X.columns), len(base_models) + 2, best_model_name,
              f"${best_rmse:,.2f}", f"{best_r2:.4f}", f"${avg_prediction:,.2f}",
              f"${predictions['Tuned XGBoost']:,.2f}"]
}

summary_df = pd.DataFrame(summary)
print("\n")
summary_df